In [18]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch
import pdfplumber

In [19]:
model_name="deepset/bert-base-cased-squad2"

In [20]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=BertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
def extract_text_from_pdf(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [22]:
pdf_file='sample.pdf'

In [23]:
text=extract_text_from_pdf(pdf_file)

In [24]:
q="What is the main topic?"

In [28]:
truncated_q=q[:511]
truncated_text=text[:511]

In [29]:
inputs=tokenizer(truncated_q,truncated_text,return_tensors='pt')

In [30]:
with torch.no_grad():
    outputs = model(**inputs)

In [31]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [33]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

''

In [34]:
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

In [35]:
outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss

In [36]:
round(loss.item(), 2)

7.33